# API Mapper Demo

## Setup

In [1]:
from apimapper import config, APIMapper
from pprint import pprint

## Configuration

APIMapper needs two parameters source (details to query the API)and mapping (correspondence between fields in the original API response and the desired fields)

## SOURCE

In [31]:
## 

## MAPPING

### Direct mapping from JSON response to desired fieldnames in wrapped response

### DBPedia
Direct 1-1 Mapping from a field called "results" in the response

In [38]:
DBPEDIA_SOURCE = {config.URL: 'http://lookup.dbpedia.org/api/search/PrefixSearch?',
                  config.QUERY_FIELD: 'QueryString'}

DBPEDIA_MAP = {config.RESULT: 'results',
               config.DIRECT: {'label': 'label',
                               'uri': 'uri'}}

api = APIMapper(DBPEDIA_SOURCE, DBPEDIA_MAP)
res = api.fetch_results('Shania')
pprint(res)


[{'label': 'Shania Twain', 'uri': 'http://dbpedia.org/resource/Shania_Twain'},
 {'label': 'Shania Twain (album)',
  'uri': 'http://dbpedia.org/resource/Shania_Twain_(album)'},
 {'label': 'Shania Twain discography',
  'uri': 'http://dbpedia.org/resource/Shania_Twain_discography'},
 {'label': 'Live (Shania Twain video)',
  'uri': 'http://dbpedia.org/resource/Live_(Shania_Twain_video)'},
 {'label': 'List of awards received by Shania Twain',
  'uri': 'http://dbpedia.org/resource/List_of_awards_received_by_Shania_Twain'}]


### GND

Direct 1-1 mapping from the response

In [24]:
GND_PERSON_MAP = {config.DIRECT: {'uri': 'id',
                                  'label': 'label'}}
GND_PERSON_SOURCE = {config.URL: 'https://lobid.org/gnd/search',
                     config.QUERY_FIELD: 'q',
                     config.PAYLOAD: {'format':'json:suggest',
                                      'filter': 'type:Person'}}
gnd_api = APIMapper(GND_PERSON_SOURCE, GND_PERSON_MAP)
res = gnd_api.fetch_results('Pratchett')
pprint(res)

[{'label': 'Pratchett, Terry | 1948-2015 | Journalist, Schriftsteller',
  'uri': 'https://d-nb.info/gnd/119252880'},
 {'label': 'Pratchett, Rhianna | 1976- | Journalistin, Drehbuchautorin',
  'uri': 'https://d-nb.info/gnd/1037047303'},
 {'label': 'Schmidt, Sabine | [19XX]- | Anglistin, Übersetzerin',
  'uri': 'https://d-nb.info/gnd/1054048231'},
 {'label': 'Brehnkmann, Peder', 'uri': 'https://d-nb.info/gnd/13788365X'},
 {'label': 'Pratchett, Lawrence', 'uri': 'https://d-nb.info/gnd/15259440X'},
 {'label': 'Pratchett, Tracey', 'uri': 'https://d-nb.info/gnd/1164173618'},
 {'label': 'Pratchett, Terry', 'uri': 'https://d-nb.info/gnd/185977111'},
 {'label': 'Pratchett, Morgan', 'uri': 'https://d-nb.info/gnd/1150939907'},
 {'label': 'Pratchett, Rhianna', 'uri': 'https://d-nb.info/gnd/140745769'},
 {'label': 'Pratchett, Morgan S.', 'uri': 'https://d-nb.info/gnd/109496011X'}]


## Applying simple ruules on the response values

### VIAF 

- Direct 1-1 mapping from a field called "result"
- Constructing URI from the response field called VIAFID. i.e., 
```
{... 
'result': {[... {...'displayForm':'Pratchett, Terry, 1948-2015','uri': '41318026'} ...]}
}
```
becomes
```
[...{'label': 'Pratchett, Terry, 1948-2015',
  'uri': 'http://www.viaf.org/viaf/76382712'}, ...]
```

In [36]:
VIAF_PERSON_MAP = {config.RESULT: 'result',
                   config.FILTER: {'nametype': 'personal'},
                   config.DIRECT: {'label': 'displayForm'},
                   config.RULES: {'uri': {config.RULE: '"http://www.viaf.org/viaf/{p1}"',
                                          config.FIELDS: {'p1': 'viafid'}}}}

    
VIAF_PERSON_SOURCE =  {config.URL: 'http://www.viaf.org/viaf/AutoSuggest',
                       config.QUERY_FIELD: 'query'}
           
viaf_api = APIMapper(VIAF_PERSON_SOURCE, VIAF_PERSON_MAP, timeout=1)
res = viaf_api.fetch_results('Pratchett')
pprint(res)

[{'label': 'Pratchett, Terry, 1948-2015',
  'uri': 'http://www.viaf.org/viaf/76382712'},
 {'label': 'Pratchett, Rhianna', 'uri': 'http://www.viaf.org/viaf/41318026'},
 {'label': 'Pratchett, Lawrence', 'uri': 'http://www.viaf.org/viaf/37216918'},
 {'label': 'Pratchett, Marina A', 'uri': 'http://www.viaf.org/viaf/105358095'},
 {'label': 'Pratchett, Tracey',
  'uri': 'http://www.viaf.org/viaf/74148449587015690502'},
 {'label': 'Pratchett, Morgan', 'uri': 'http://www.viaf.org/viaf/304310389'}]


## GND and VIAF for person

Combined response:
- URI: `GND:id, VIAF:http://www.viaf.org/viaf/AutoSuggest/{viafid}`
- LABEL: `GND:label, VIAF:displayForm`


In [37]:
results = []
for api in (gnd_api, viaf_api):            
    res = api.fetch_results('Pratchett')
    results.extend(res)

pprint(results)

[{'label': 'Pratchett, Terry | 1948-2015 | Journalist, Schriftsteller',
  'uri': 'https://d-nb.info/gnd/119252880'},
 {'label': 'Pratchett, Rhianna | 1976- | Journalistin, Drehbuchautorin',
  'uri': 'https://d-nb.info/gnd/1037047303'},
 {'label': 'Brehnkmann, Peder', 'uri': 'https://d-nb.info/gnd/13788365X'},
 {'label': 'Schmidt, Sabine | [19XX]- | Anglistin, Übersetzerin',
  'uri': 'https://d-nb.info/gnd/1054048231'},
 {'label': 'Pratchett, Lawrence', 'uri': 'https://d-nb.info/gnd/15259440X'},
 {'label': 'Pratchett, Tracey', 'uri': 'https://d-nb.info/gnd/1164173618'},
 {'label': 'Pratchett, Terry', 'uri': 'https://d-nb.info/gnd/185977111'},
 {'label': 'Pratchett, Morgan', 'uri': 'https://d-nb.info/gnd/1150939907'},
 {'label': 'Pratchett, Rhianna', 'uri': 'https://d-nb.info/gnd/140745769'},
 {'label': 'Pratchett, Morgan S.', 'uri': 'https://d-nb.info/gnd/109496011X'},
 {'label': 'Pratchett, Terry, 1948-2015',
  'uri': 'http://www.viaf.org/viaf/76382712'},
 {'label': 'Pratchett, Rhianna

## Video Sources

### IMVDB

In [2]:
import logging
logging.basicConfig(level=logging.ERROR)
IMVDB_SOURCE = {config.URL: 'https://imvdb.com/api/v1/search/videos?',
                  config.QUERY_FIELD: 'q'}

IMVDB_MAP = {config.RESULT: 'results',
             config.DIRECT: {'label': 'song_title','year': 'year',
                             'uri': 'url'},
             config.RULES: {'source_name': {config.RULE: '"IMVDB"'},
                            'label': {config.RULE:"'\"{p1}\"' + ' by '+','.join([artist['name'] for artist in {artists}])",
                                      config.FIELDS: { 'artists': 'artists','p1': 'song_title',}}}
            }


In [3]:
api = APIMapper(IMVDB_SOURCE, IMVDB_MAP, timeout=1)
api.fetch_results('common people')

[{'label': '"The People" by Common',
  'year': 2007,
  'uri': 'https://imvdb.com/video/common/the-people',
  'source_name': 'IMVDB'},
 {'label': '"Common People" by Pulp',
  'year': 1995,
  'uri': 'https://imvdb.com/video/pulp/common-people',
  'source_name': 'IMVDB'},
 {'label': '"Love of the Common People" by Paul Young',
  'year': None,
  'uri': 'https://imvdb.com/video/paul-young-1/love-of-the-common-people',
  'source_name': 'IMVDB'},
 {'label': '"Celebrate" by Common',
  'year': 2011,
  'uri': 'https://imvdb.com/video/common/celebrate',
  'source_name': 'IMVDB'},
 {'label': '"Be" by Common',
  'year': 2005,
  'uri': 'https://imvdb.com/video/common/be',
  'source_name': 'IMVDB'},
 {'label': '"Nothing" by Common Holly',
  'year': 2017,
  'uri': 'https://imvdb.com/video/common-holly/nothing',
  'source_name': 'IMVDB'},
 {'label': '"Sweet" by Common',
  'year': '',
  'uri': 'https://imvdb.com/video/common/sweet',
  'source_name': 'IMVDB'},
 {'label': '"The Light" by Common',
  'year'

### OMDb

In [5]:
import os
OMDB_API_KEY = os.environ.get('OMDB_API_KEY')
# key is saved in $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
print(OMDB_API_KEY)

8219a74c


In [6]:
OMDB_SOURCE = {config.URL: 'http://www.omdbapi.com/',
               config.QUERY_FIELD: 's',
              config.PAYLOAD: {'apikey':OMDB_API_KEY}}

OMDB_MAP = {config.RESULT: 'Search',
             config.DIRECT: {'label': 'Title', 'year': 'Year'},
             config.RULES: {'source_name': {config.RULE: '"OMDB"'},
                            'uri': {config.RULE:'"https://www.imdb.com/title/{imdbID}"',
                                    config.FIELDS: {'imdbID': 'imdbID'}}}
            }


In [7]:
api = APIMapper(OMDB_SOURCE, OMDB_MAP, timeout=1)
res =api.fetch_results('Vertigo')
pprint(res)

[{'label': 'Vertigo',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt0052357',
  'year': '1958'},
 {'label': "Cats Don't Have Vertigo",
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt2886926',
  'year': '2014'},
 {'label': 'Vertigo 2005: U2 Live from Chicago',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt0488664',
  'year': '2005'},
 {'label': 'Obsessed with Vertigo',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt0192387',
  'year': '1997'},
 {'label': 'Vertigo',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt5433782',
  'year': '2016'},
 {'label': 'Angel Guts 5: Red Vertigo',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt0224236',
  'year': '1988'},
 {'label': 'Love Vertigo',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt0279494',
  'year': '2001'},
 {'label': 'U2 - Vertigo 2005: Live from Milan',
  'source_name': 'OMDB',
  'uri': 'https://www.imdb.com/title/tt09